# Automatic Recognition of Vehicles at Intersections and Update of Moving Vehicles
 


### Monitoring Execution
Using the time library we will monitor our programms execution.

In [1]:
#Start counting the time execution of our programm.
%run timer_class.ipynb
timer = Timer()
timer.start()

Firstly, we will create a function that will check the existense of all necessary libraries.So as to ensure a stable execution of our programm.

In [2]:
%pip freeze >requirements.txt
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys 
import os

import numpy as np
import cv2 as cv
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import skimage.io
import cython
import random
import keras
import h5py
import pycocotools

In [4]:
print(tf.__version__)
print(keras.__version__)

2.12.0
2.12.0


In [5]:
#Now we are sure for the existence of the libraries. We can safely proceed with importing them.
# import numpy as np
# import cv2 
# import tensorflow as tf
# import matplotlib as plt
# import pandas as pd


import warnings 
warnings.filterwarnings("ignore")


In [6]:
#Stop counting time see how much time it took for the libraries to get imported.
timer.stop()

Elapsed time is: 19.24 seconds


In [7]:
#Have a variable keep the root dir of the project and one for the current directory.
root_dir = os.path.abspath("../")
cur_dir = os.getcwd()
root_dir,cur_dir

('c:\\Users\\saraf\\Project_Master_Thesis',
 'c:\\Users\\saraf\\Project_Master_Thesis\\Project')

In [8]:
#Import the Mask RCNN 2.1 using the local version we downloaded from github
lib_path = os.path.join(root_dir,"mrcnn")
#print (f"mrcnn path: {lib_path} ")
sys.path.insert(1,lib_path) 
import utils
import model as modellib
import visualize


In [9]:
#Import the coco directory
import coco
%matplotlib inline

We will create different variables holding the paths towards important directories, such as logs and video/images for evaluation.

In [10]:
#Log dir path and create the log directory
log_dir = os.path.join(root_dir,"logs")
os.makedirs(log_dir,exist_ok= True)
log_dir

'c:\\Users\\saraf\\Project_Master_Thesis\\logs'

In [11]:
#local path for the rcnn coco weights
coco_model_path = os.path.join('', "mask_rcnn_coco.h5")
coco_model_path

'mask_rcnn_coco.h5'

In [12]:
#path pointing at the images folder
eva_image_dir = os.path.join(root_dir,"evaluation_Images")
os.makedirs(eva_image_dir,exist_ok= True)
eva_image_dir

'c:\\Users\\saraf\\Project_Master_Thesis\\evaluation_Images'

In [13]:
#same for evaluation video, since that's the end goal.
eva_video_dir = os.path.join(root_dir,"evaluation_Video")
os.makedirs(eva_video_dir,exist_ok= True)
eva_video_dir

'c:\\Users\\saraf\\Project_Master_Thesis\\evaluation_Video'

In [14]:
#!tf_upgrade_v2  --intree C:\Users\saraf\Project_Master_Thesis\Project\mrcnn --inplace --reportfile report.txt

In [15]:
#create the inference class. We will use that to infer the Mask rcnn model.
class InferenceConfig(coco.CocoConfig):
    # Batch size =  GPU_Count * Images_per_GPU
    # since we have 1 GPY and 1 image at the time both will be 1.
    GPU_count = 1
    image_per_GPU = 1


In [16]:
#Supposing all went well we should be able to successfully infer our model
config =  InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GPU_count                      1
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE             

### We have sucessfully infered the Mask R-Cnn archetype
Above we have a list of all hyper parameters of the model. Most important among them are, the shape of Mask [28,28] which instead of [256,256] because it is trained in the COCO dataset. The possible class predictions are 81. What we are interested in is car classification in our images.

In [19]:
#Create the model object 
"""
        mode: Either "training" or "inference"
        config: A Sub-class of the Config class
        model_dir: Directory to save training logs and trained weights
"""
model = modellib.MaskRCNN(mode = "inference", model_dir="our_model.hy", config = config)

ValueError: Exception encountered when calling layer "mrcnn_detection" (type DetectionLayer).

in user code:

    File "c:\Users\saraf\Project_Master_Thesis\mrcnn\model.py", line 794, in call  *
        detections_batch = utils.batch_slice(
    File "c:\Users\saraf\Project_Master_Thesis\mrcnn\utils.py", line 699, in batch_slice  *
        output_slice = graph_fn(*inputs_slice)
    File "c:\Users\saraf\Project_Master_Thesis\mrcnn\model.py", line 687, in refine_detections_graph  *
        indices = tf.stack([tf.range(probs.shape[0]), class_ids], axis=1)

    ValueError: None values not supported.


Call arguments received by layer "mrcnn_detection" (type DetectionLayer):
  • inputs=['tf.Tensor(shape=(2, None, 4), dtype=float32)', 'tf.Tensor(shape=(1, None, 81), dtype=float32)', 'tf.Tensor(shape=(1, None, 81, 4), dtype=float32)', 'tf.Tensor(shape=(None, None), dtype=float32)']

In [ ]:
#Lets load the pre-trained weights
model.load_weights('mask_rcnn_coco.h5', by_name =True)

In [ ]:
# end the time
timer.stop()